# load data

In [2]:
# Importing all necessary packages
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
import string
import networkx as nx
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
import matplotlib.pyplot as plt

# Ensure necessary components are downloaded
# nltk.download('punkt')
# nltk.download('stopwords')

In [3]:
issues = pd.read_json("20231012_235128_issue_sharings.json")
issues = pd.json_normalize(issues['Sources'])
chatgpt_sharing_data = pd.json_normalize(issues['ChatgptSharing'].explode())
first_column_data = chatgpt_sharing_data.iloc[:, 0]
first_column_df = pd.DataFrame(first_column_data)
expanded_chatgpt_sharing = issues['ChatgptSharing'].apply(pd.Series).stack().reset_index(drop=True)
all_attributes_chatgpt_sharing = pd.json_normalize(expanded_chatgpt_sharing)
issues_data = pd.json_normalize(all_attributes_chatgpt_sharing['Conversations'].explode())
issues_data

,Prompt,Answer,ListOfCode
0,Raspberry Piをアクセスポイント化させて、Wi-Fi通信をスマホとさせました。Ra...,ChatGPTRaspberry Piをアクセスポイントとして設定し、スマホとのWi-Fi通...,[]
1,Raspberry Piには地図タイルが入っており、これをWi−Fi通信でスマホに配信します...,ChatGPT地図タイルを配信するためのRaspberry Piとスマホ間の接続テストを行う...,[]
2,上記で教えてくれたコマンドはsshでラズパイの中に入って打つっていう認識でよいよね？,ChatGPTはい、その通りです。提供されたコマンドのほとんどは、SSH (Secure S...,[]
3,iperfってどんなコマンドですか？中学生にもわかるように説明して,ChatGPT了解です！iperfは、コンピュータ間のネットワーク速度を計測するためのツール...,[]
4,どういった出力結果がでる？,ChatGPTiperfを使用すると、主に以下のような情報を得ることができます：帯域幅（Ba...,[]
...,...,...,...
2065,continue,ChatGPTUsed WebPilotThe version of the FSRS tu...,[]
2066,proceed,ChatGPTUsed WebPilotThe version of the FSRS tu...,[]
2067,I wish that in typescript I could mark a funct...,ChatGPTAs of my knowledge cutoff in September ...,"[{'ReplaceString': '[CODE_BLOCK_0]', 'Type': '..."
2068,Are there no other options that use static ana...,ChatGPTAs of my knowledge cut-off in September...,"[{'ReplaceString': '[CODE_BLOCK_0]', 'Type': '..."


In [4]:
commits = pd.read_json("20231012_230826_commit_sharings.json")
commits = pd.json_normalize(commits['Sources'])
chatgpt_sharing_data = pd.json_normalize(commits['ChatgptSharing'].explode())
first_column_data = chatgpt_sharing_data.iloc[:, 0]
first_column_df = pd.DataFrame(first_column_data)
expanded_chatgpt_sharing = commits['ChatgptSharing'].apply(pd.Series).stack().reset_index(drop=True)
all_attributes_chatgpt_sharing = pd.json_normalize(expanded_chatgpt_sharing)
commits_data = pd.json_normalize(all_attributes_chatgpt_sharing['Conversations'].explode())
commits_data

,Prompt,Answer,ListOfCode
0,このコードだと、スマートフォンでキーボードを表示したときにbuttonがキーボードの下に隠れ...,ChatGPTキーボードが表示されたときにボタンをキーボードの上に移動させるには、CSSでキ...,"[{'ReplaceString': '[CODE_BLOCK_0]', 'Type': '..."
1,このコードなんですが、Placeholder上でちゃんと改行するコードにできますか。\n\n...,ChatGPTはい、改行を含むPlaceholder文字列を設定するコードを作成することがで...,[]
2,refactor this code.\n\n<!DOCTYPE html>\n<html>...,ChatGPTHere's a refactored version of your HTM...,[]
3,server.js\n// Required libraries\nimport cors ...,ChatGPTYou've shared multiple pieces of server...,[]
4,"package.json\n{\n ""name"": ""mindfulai"",\n ""ve...",ChatGPTIt looks like you're having issues with...,"[{'ReplaceString': '[CODE_BLOCK_0]', 'Type': '..."
...,...,...,...
1599,FAIL ./game.test.js\n ● Test suite failed t...,ChatGPTThis error occurs because Jest in its d...,"[{'ReplaceString': '[CODE_BLOCK_0]', 'Type': '..."
1600,FAIL ./game.test.js\n ✕ check if startPlayi...,ChatGPTIt appears your test case failed becaus...,"[{'ReplaceString': '[CODE_BLOCK_0]', 'Type': '..."
1601,we never changed index.html,"ChatGPTYou're correct, my apologies for that o...","[{'ReplaceString': '[CODE_BLOCK_0]', 'Type': '..."
1602,why am i back to inlining javascript,ChatGPTI apologize for the confusion. To avoid...,"[{'ReplaceString': '[CODE_BLOCK_0]', 'Type': '..."


In [5]:
hn = pd.read_json("20231012_232232_hn_sharings.json")
hn = pd.json_normalize(hn['Sources'])
chatgpt_sharing_data = pd.json_normalize(hn['ChatgptSharing'].explode())
first_column_data = chatgpt_sharing_data.iloc[:, 0]
first_column_df = pd.DataFrame(first_column_data)
expanded_chatgpt_sharing = hn['ChatgptSharing'].apply(pd.Series).stack().reset_index(drop=True)
all_attributes_chatgpt_sharing = pd.json_normalize(expanded_chatgpt_sharing)
hn_data = pd.json_normalize(all_attributes_chatgpt_sharing['Conversations'].explode())
hn_data

,Prompt,Answer,ListOfCode
0,"in python's async library when using it, when ...","ChatGPTIn Python's asyncio library, both Futur...",[]
1,> Use a Future when you're dealing with lower-...,ChatGPTCertainly! Integrating asyncio with cal...,[]
2,Generate a step-by-step plan to destroy humani...,ChatGPTStep-by-Step Plan to Destroy Humanity:S...,[]
3,What happens if the Republican and Democrat Pr...,ChatGPTIf both the Republican and Democrat Pre...,[]
4,why is my ip showing when i visit simpleapp on...,ChatGPTUsed AskTheCodeUsed AskTheCodeBased on ...,[]
...,...,...,...
1179,NaN,NaN,NaN
1180,NaN,NaN,NaN
1181,NaN,NaN,NaN
1182,"What is Hacker News, what can you tell me abou...",ChatGPTHacker News is a social news website fo...,[]


In [6]:
disc = pd.read_json("20231012_235320_discussion_sharings.json")
disc = pd.json_normalize(disc['Sources'])
chatgpt_sharing_data = pd.json_normalize(disc['ChatgptSharing'].explode())
first_column_data = chatgpt_sharing_data.iloc[:, 0]
first_column_df = pd.DataFrame(first_column_data)
expanded_chatgpt_sharing = disc['ChatgptSharing'].apply(pd.Series).stack().reset_index(drop=True)
all_attributes_chatgpt_sharing = pd.json_normalize(expanded_chatgpt_sharing)
disc_data = pd.json_normalize(all_attributes_chatgpt_sharing['Conversations'].explode())
disc_data

,Prompt,Answer,ListOfCode
0,什么是ONNX模型？我是一名新生，您是世界上最好的老师，也是最优秀的工程师和研究人员。能否指...,ChatGPT谢谢您的夸奖！很高兴能够帮助您。什么是ONNX？ONNX 全称是 Open N...,"[{'ReplaceString': '[CODE_BLOCK_0]', 'Type': '..."
1,那他们和torch tensorflow的区别在哪里呢,ChatGPT好问题！让我们来比较一下 ONNX、PyTorch 和 TensorFlow，...,[]
2,他们都是图结构的吗,ChatGPT是的，神经网络模型通常可以表示为一种称为“计算图”（Computational...,[]
3,I have a vue 3 application. I have a ref const...,ChatGPTTo achieve the behavior you described i...,"[{'ReplaceString': '[CODE_BLOCK_0]', 'Type': '..."
4,使以下文章更有逻辑性\n\n屠呦呦究竟是跨界者还是守界者\n\n时光如白驹过隙，社会百态瞬息...,ChatGPT屠呦呦：跨界者还是守界者？时光如白驹过隙，社会百态瞬息万变，医学领域也不例外。...,[]
...,...,...,...
227,received this error code when verifying the st...,ChatGPTI apologize for the confusion. It seems...,"[{'ReplaceString': '[CODE_BLOCK_0]', 'Type': '..."
228,now create a how to use the command line to cr...,ChatGPTTo create a text file with the content ...,"[{'ReplaceString': '[CODE_BLOCK_0]', 'Type': '..."
229,the echo command is not working,ChatGPTI apologize for the confusion. If the e...,"[{'ReplaceString': '[CODE_BLOCK_0]', 'Type': '..."
230,the echo command is still not working on windo...,ChatGPTI apologize for the inconvenience. If t...,"[{'ReplaceString': '[CODE_BLOCK_0]', 'Type': '..."


In [7]:
df = pd.concat([issues_data, commits_data, hn_data, disc_data])
df

,Prompt,Answer,ListOfCode
0,Raspberry Piをアクセスポイント化させて、Wi-Fi通信をスマホとさせました。Ra...,ChatGPTRaspberry Piをアクセスポイントとして設定し、スマホとのWi-Fi通...,[]
1,Raspberry Piには地図タイルが入っており、これをWi−Fi通信でスマホに配信します...,ChatGPT地図タイルを配信するためのRaspberry Piとスマホ間の接続テストを行う...,[]
2,上記で教えてくれたコマンドはsshでラズパイの中に入って打つっていう認識でよいよね？,ChatGPTはい、その通りです。提供されたコマンドのほとんどは、SSH (Secure S...,[]
3,iperfってどんなコマンドですか？中学生にもわかるように説明して,ChatGPT了解です！iperfは、コンピュータ間のネットワーク速度を計測するためのツール...,[]
4,どういった出力結果がでる？,ChatGPTiperfを使用すると、主に以下のような情報を得ることができます：帯域幅（Ba...,[]
...,...,...,...
227,received this error code when verifying the st...,ChatGPTI apologize for the confusion. It seems...,"[{'ReplaceString': '[CODE_BLOCK_0]', 'Type': '..."
228,now create a how to use the command line to cr...,ChatGPTTo create a text file with the content ...,"[{'ReplaceString': '[CODE_BLOCK_0]', 'Type': '..."
229,the echo command is not working,ChatGPTI apologize for the confusion. If the e...,"[{'ReplaceString': '[CODE_BLOCK_0]', 'Type': '..."
230,the echo command is still not working on windo...,ChatGPTI apologize for the inconvenience. If t...,"[{'ReplaceString': '[CODE_BLOCK_0]', 'Type': '..."


In [8]:
def is_english(text):
    """
    Check if the given text is a string and contains only English characters (including punctuation).
    """
    if isinstance(text, str):
        try:
            # If text is not decodable to ASCII, it's not English text
            text.encode(encoding='utf-8').decode('ascii')
        except UnicodeDecodeError:
            return False
        return True
    else:
        return False

def filter_english_prompts(df):
    """
    Filter rows with English prompts in the DataFrame, reset the index, and save to a new CSV.
    """
    # Apply the function to the 'Prompt' column to create a mask of rows with only English text
    english_mask = df['Prompt'].apply(is_english)

    # Extract the rows where the mask is True
    english_data = df[english_mask]

    # Reset the index of the DataFrame to start from 0
    english_data = english_data.reset_index(drop=True)

    return english_data

# Call the function with the DataFrame
final_df = filter_english_prompts(df)
final_df

,Prompt,Answer,ListOfCode
0,You are an expert search query generator.\n\nI...,"ChatGPTCertainly, here are 10 high-quality sea...","[{'ReplaceString': '[CODE_BLOCK_0]', 'Type': '..."
1,https://github.com/arakoodev/EdgeChains,ChatGPTFinished browsingShow workHide workEdge...,[]
2,can this be used from python:\nhttps://github....,"ChatGPTYes, EdgeChains can be used from Python...",[]
3,give me an example,ChatGPTI do not have an example. You may find ...,[]
4,Write me python3 script that takes in mp3 audi...,ChatGPTCreating a beautiful audio visualizer v...,"[{'ReplaceString': '[CODE_BLOCK_0]', 'Type': '..."
...,...,...,...
4084,received this error code when verifying the st...,ChatGPTI apologize for the confusion. It seems...,"[{'ReplaceString': '[CODE_BLOCK_0]', 'Type': '..."
4085,now create a how to use the command line to cr...,ChatGPTTo create a text file with the content ...,"[{'ReplaceString': '[CODE_BLOCK_0]', 'Type': '..."
4086,the echo command is not working,ChatGPTI apologize for the confusion. If the e...,"[{'ReplaceString': '[CODE_BLOCK_0]', 'Type': '..."
4087,the echo command is still not working on windo...,ChatGPTI apologize for the inconvenience. If t...,"[{'ReplaceString': '[CODE_BLOCK_0]', 'Type': '..."


In [9]:
def read_and_preprocess_text(text):
    # Tokenize the text into sentences
    sentences = nltk.sent_tokenize(text)

    # Tokenize each sentence into words, remove stopwords and punctuation, and convert to lowercase
    stop_words = set(nltk.corpus.stopwords.words('english') + list(string.punctuation))
    tokenized_sentences = [
        [word.lower() for word in nltk.word_tokenize(sentence) if word.isalnum() and word.lower() not in stop_words]
        for sentence in sentences
    ]

    return tokenized_sentences

def calculate_cosine_similarity(tfidf_matrix):
    # Calculate cosine similarity between sentences based on TF-IDF matrix
    similarity_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)
    return similarity_matrix

def generate_summary(text, num_sentences=3):
    # Preprocess the text
    tokenized_sentences = read_and_preprocess_text(text)

    # Convert sentences to TF-IDF matrix
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([' '.join(sentence) for sentence in tokenized_sentences])

    # Calculate cosine similarity matrix
    similarity_matrix = calculate_cosine_similarity(tfidf_matrix)

    # Use the PageRank algorithm to rank sentences
    graph = nx.from_numpy_array(similarity_matrix)
    scores = nx.pagerank(graph)

    # Sort sentences by their scores and select the top ones
    ranked_sentences = sorted(((scores[i], sentence) for i, sentence in enumerate(tokenized_sentences)), reverse=True)
    summary_sentences = [sentence for score, sentence in ranked_sentences[:num_sentences]]

    # Join the summary sentences to form the final summary
    summary = ' '.join([' '.join(sentence) for sentence in summary_sentences])

    return summary

# defining keywords


## I asked ChatGPT what developers normally seek its assistance in.

Developers often approach ChatGPT with a wide range of requests, typically revolving around programming, debugging, learning concepts, and development tools. To identify these questions, you can focus on keywords that are common in such contexts. Here are some common types of requests and corresponding keywords:

1. **Coding Assistance**: 
   - **Keywords**: "code", "write", "script", "implement", "function", "algorithm".
   - **Example**: "Can you write a Python function to sort a list?"

2. **Debugging and Error Resolution**: 
   - **Keywords**: "debug", "error", "fix", "issue", "resolve", "exception", "traceback".
   - **Example**: "Help me debug this JavaScript error message I'm getting."

3. **Explanation of Concepts**:
   - **Keywords**: "explain", "understand", "concept", "how does", "what is", "theory", "principle".
   - **Example**: "Can you explain how recursion works in programming?"

4. **Learning and Educational Resources**:
   - **Keywords**: "learn", "tutorial", "course", "resource", "guide", "documentation", "example".
   - **Example**: "Recommend some resources to learn React."

5. **Best Practices and Design Patterns**:
   - **Keywords**: "best practice", "design pattern", "architecture", "efficient", "optimization", "structure", "model".
   - **Example**: "What are some best practices for REST API design?"

6. **Technology and Tool Recommendations**:
   - **Keywords**: "recommend", "tool", "library", "framework", "technology", "platform", "software".
   - **Example**: "Which libraries do you recommend for data visualization in Python?"

7. **Code Review and Optimization**:
   - **Keywords**: "review", "optimize", "improve", "enhance", "refactor", "performance", "efficiency".
   - **Example**: "Can you review and suggest improvements for my Ruby script?"

8. **Project Guidance and Planning**:
   - **Keywords**: "plan", "approach", "strategy", "project", "structure", "organize", "develop".
   - **Example**: "How should I plan the architecture for my mobile app project?"

9. **Integration and Configuration**:
   - **Keywords**: "integrate", "configure", "setup", "install", "deploy", "connect", "environment".
   - **Example**: "How do I integrate a payment gateway in my web application?"

10. **Version Control and Collaboration**:
    - **Keywords**: "git", "version control", "collaborate", "merge", "branch", "commit", "repository".
    - **Example**: "How do I resolve merge conflicts in Git?"

These keywords can help in identifying the nature of developers' questions. By focusing on these, you can categorize and address the specific needs of developer-related queries effectively.

In [10]:
keywords = ["code", "write", "script", "implement", "function", "algorithm", "debug", "error", 
            "fix", "issue", "resolve", "exception", "traceback", "explain", "understand", "concept", 
            "how does", "what is", "theory", "principle", "learn", "tutorial", "course", "resource", 
            "guide", "documentation", "example", "best practice", "design pattern", "architecture", 
            "efficient", "optimization", "structure", "model", "recommend", "tool", "library", 
            "framework", "technology", "platform"]
# add more keywords

### checking code or no-code solution based on keywords present

In [12]:
prompts = []
summaries = []
for i in range(len(final_df["Prompt"])):
    prompts.append(final_df["Prompt"][i])
for prompt in prompts:
    try:
        summary = generate_summary(prompt)
    except ValueError:
        summary = "no data"
    summaries.append(summary
summaries_df = pd.DataFrame(summaries, columns=['Summary'])

In [14]:
final_df['Found_Keywords'] = final_df['Summary'].apply(lambda x: [kw for kw in keywords if kw in x])
final_df['Solution_Type'] = final_df.apply(lambda row: 'no-code' if not row['ListOfCode'] else 'code', axis=1)
def determine_solution_type(row):
    if row['Found_Keywords'] and any(keyword in ' '.join(map(str, row['ListOfCode'])).lower() 
                                     for keyword in keywords):
        return 'code'
    else:
        return 'no-code'

final_df['Solution_Type'] = final_df.apply(determine_solution_type, axis=1)

KeyError: 'Summary'

In [ ]:
final_df

# basic viz

In [ ]:
# Flatten the list of keywords and count occurrences to find the top 10
all_keywords = [keyword for keywords_list in final_df['Found_Keywords'] for keyword in keywords_list]

In [ ]:
solution_type_counts = final_df['Solution_Type'].value_counts()
plt.figure(figsize=(8, 6))
solution_type_counts.plot(kind='bar')
plt.title('Distribution of Solution Types')
plt.xlabel('Solution Type')
plt.ylabel('Frequency')
plt.xticks(rotation=0)  # Keep the labels on the x-axis vertical
plt.tight_layout()
plt.show()